## Introducción a Pyspark (II): Creación de un modelo sencillo

En este notebook (que debes ejecutar en Databricks) vamos a crear un modelo muy sencillo de regresión lineal para que te familiarices en cómo se hace. Verás que el proceso es casi idéntico al realizado con Python con pequeñas matizaciones, algunas no tan pequeñas. Después te animo a que hagas la práctica obligatoria, eso sí con el apoyo de la documentación:

Las genéricas (Guia de uso y de APIs, ojo de las funciones y su uso no de API Rest)
- https://spark.apache.org/docs/latest/api/python/user_guide/index.html
- https://spark.apache.org/docs/latest/api/python/reference/index.html

La específica para MLlib:
- https://spark.apache.org/docs/latest/api/python/reference/pyspark.ml.html



### Objetivo de negocio

Tenemos un problema aparentemente sencillo, crea un modelo simple de predicción del salario de una compalía a partir de la experiencia y de la edad de un empleado. Esta vez objetivo y modelo coinciden.

### Lectura de datos y primer vistazo

Antes de poder hacer nada, recuerda que en Spark es necesario abrir una sesión.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Modelo').getOrCreate()

In [ ]:
spark

Vamos a utilizar una de las tablas o ficheros csv que ya cargamos al comienzo de las sesiones prácticas. Comprobemos primero que sigue aquí:

In [ ]:
display(dbutils.fs.ls("dbfs:/FileStore/tables/"))

Ahí está, la test1.csv. Creemos un dataframe con ella.

In [ ]:
training = spark.read.csv("dbfs:/FileStore/tables/test1.csv", inferSchema=True, header=True)

Y echamos un vistazo que en este caso es verla entera

In [ ]:
training.show()

El target es claramente Salary:

In [ ]:
target = "Salary"

### Train-test split, MiniEDA

Aunque no tiene mucho sentido por la cantidad de datos hacer un split, veamos como se hace un train-test split:

In [ ]:
train_set, test_set = training.randomSplit([0.65,0.35], seed = 42) # Usamos estos valores de split para que salga algo minimamente usable

In [ ]:
display(train_set.show())

In [ ]:
display(test_set.show())

Respecto al miniEDA, este dataset no requiere mucho pero aquí se abre un pequeño debate... En Spark no vas a hacer un miniEDA al uso, no al menos con la parte de visualización. En general, si estás usando Spark es que estamos hablando de grandes volúmenes de datos (en absoluto o por unidad de tiempo)... ¿Cuál sería la forma de proceder? Hacer un miniEDA de una muestra representativa del DataFrame de Train.

¿Cómo?
- Hacer un RandomSplit al 20,80. Quedarse con la parte del 20, pasarla a Pandas y trabajar como hemos venido trabajando hasta aquí. ¿Pasarla a Pandas?

Sí, supongamos que train_set es ya nuestra submuestra del train que podemos manejar en memoria en nuestros equipos (fuera del cluster):

In [ ]:
df_minieda = train_set.toPandas()
print(type(df_minieda))

Ya es un dataframe pandas, eso quiere decir que no tiene un RDD por debajo, no está particionado y por tanto aunque lo ejecutes en un cluster no saldrá del nodo driver para ser analizado.

In [ ]:
df_minieda

Nos quedamos con las dos features que no son el target

In [ ]:
features = ["age","Experience"]

### Tratamiento y generación de Features

El tratamiento de features hay que hacerlo y para ello tendrás que emplear las funciones tal y como vimos en la parte dedicada a la sintáxis básica (sí tendrás que ver como hacer una estandarizacion, pero ya tienes suficiente capacidad como para verlo en la documentación)

**IMPORTANTE:**
Pero **la generación de features (su tratamiento final) en Pyspark tiene un punto especial e importante**. Todos los modelos esperan una única columna cuyos valores son un vector con los valores de las features del modelo (y por tanto con el mismo orden para todas las instancias)

Tendremos que agrupar nuestras variables independientes de forma que queden todas en una columna y dentro de una lista, por lo que crearemos un vector de ensamblaje o "vector assembler", de tal modo que queden así esas variables independientes:
- [Age, Experience]

Lo que haremos con estas dos, será tratarlas como una nueva variable independiente:
- [Age, Experience] ----> nueva_variable_independiente

In [ ]:
from pyspark.ml.feature import VectorAssembler 

In [ ]:
features = "Independent features"
feature_assembler = VectorAssembler(inputCols=['age', 'Experience'], outputCol= features) 

Como puedes ver eso no hace nada sobre el dataframe, ahora hay que aplicarlo

In [ ]:
output_train = feature_assembler.transform(train_set)

Veremos que se crea una nueva columna cuyos valores se corresponden a unos array con el contenido de aquellas variables independientes que hemos agrupado. Esto será nuestro input feature o lo que solíamos definir como train.

In [ ]:
output_train.show()

Seleccionamos las columnas que nos interesan para nuestro modelo: el train (Independent Features) y el test (Salary)

In [ ]:
finalized_data_train = output_train.select(features, target)
finalized_data_train.show()

Ahora preparamos el test

In [ ]:
output_test = feature_assembler.transform(test_set) # No te rayes, no estamos normalizando ni haciendo nada que dependa del train por eso se aplica al test directamente
finalized_data_test = output_test.select(features, target)
finalized_data_test.show()

### Instanciación y entrenamiento del modelo

A continuación, entrenaremos un modelo de regresión lineal (fijate que volvemos a la librería ml de pyspark)

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
regressor = LinearRegression(featuresCol= features, labelCol= target)
regressor = regressor.fit(finalized_data_train) # Al indicarle la labelCol no tengo que deshacerme del target en el train

Podemos ver los parámetros que ha entrenado el modelo (es decir los coeficientes de la regresión lineal):

In [ ]:
regressor.coefficients

Asi como el intercetpo (intercept)

In [ ]:
regressor.intercept

### Evaluación del modelo

Aquí viene otra pequeña diferencia (no tan intensa como la del vector assembler) que es bueno que tengas en cuenta. Creamos las predicciones llamando al metodo evaluate:

In [ ]:
prediction = regressor.evaluate(finalized_data_test)

Veamos de qué tipo es esa prediction:

In [ ]:
type(prediction)

Es un objeto especial, para ver las predicciones tenemos que bucear un poco más:

In [ ]:
prediction.predictions.show()

Y a cambio viene con sus métodos para obtener las métricas directamente:

In [ ]:
# Errores

prediction.meanAbsoluteError, prediction.meanSquaredError

Ya tienes todo lo necesario para seguir profundizando, si quieres, y sobre todo para terminar todo esta unidad de Spark lanzandote a por la práctica obligatoria (que vas a necesitar bucear en la documentación, pero eso es ley de vida en el mundo de la ciencia de datos)